### Check cuda availability

In [1]:
import pandas as pd
import torch
import gc
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import json
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import BaseTool
from ddgs import DDGS
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("WARNING: CUDA is NOT available. You are running on CPU.")

PyTorch Version: 2.9.1+cu126
CUDA Available: True
CUDA Version: 12.6
Device Name: NVIDIA GeForce RTX 4090


In [4]:
# ==========================================
# BLOCK 1: DATA INGESTION & SMART CHUNKING
# ==========================================
class ClinicalDataProcessor:
    def __init__(self, file_path: str):
        self.df = pd.read_csv(file_path)
        print(f"Data Loaded. Columns: {self.df.columns.tolist()}")

    def get_case_notes(self, case_num: int) -> str:
        """
        Filters dataframe by case_num and combines all history notes 
        into a single massive string.
        """
        case_df = self.df[self.df['case_num'] == case_num]
        # Combining notes with a newline separator for clarity
        combined_text = "\n".join(case_df['pn_history'].astype(str).tolist())
        return combined_text

    def smart_chunking(self, text: str, chunk_size: int = 2000, overlap_percentage: float = 0.15) -> List[str]:
        """
        Splits text while preserving context using overlap.
        Default 15% overlap falls within your 10-20% requirement.
        """
        overlap = int(chunk_size * overlap_percentage)
        
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", ". ", " ", ""],
            length_function=len
        )
        
        chunks = splitter.split_text(text)
        print(f"Text split into {len(chunks)} chunks with {overlap} char overlap.")
        return chunks

In [ ]:
# ==========================================
# BLOCK 2: MODULAR LOCAL LLM SUMMARIZER
# ==========================================
class ClinicalSummarizer:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"CUDA available on: {self.device}")

    def _load_model(self, model_id: str):
        """
        Loads a model with 8-bit quantization to fit in consumer VRAM.
        
        """
        print(f"--- Loading {model_id} (4-bit) ---")
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
        )

        # Retrieve the Hugging Face token from the environment variables
        hf_token = os.getenv('HF_TOKEN')

        try:
            # Pass the token explicitly to the tokenizer
            tokenizer = AutoTokenizer.from_pretrained(model_id, token=hf_token)
            
            # Also passing the token to the model loader
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                quantization_config=bnb_config,
                device_map='auto',
                trust_remote_code=True,
                token=hf_token
            )
            return model, tokenizer
            
        except Exception as e:
            print(f"Error loading model {model_id}: {e}")
            raise

    def _cleanup(self, model, tokenizer):
        """
        Forcefully clears VRAM to allow the next model to load.
        """
        del model
        del tokenizer
        gc.collect()
        torch.cuda.empty_cache()
        print("--- VRAM Cleared ---")

    def summarize_chunks(self, chunks: List[str], model_id: str) -> str:
        """
        Iterates through chunks, summarizes them, and returns a joined summary.
        """
        if not chunks:
            return "No text to summarize."

        model, tokenizer = self._load_model(model_id)
        
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512,
            temperature=0.1, # Low temp for factual consistency
            repetition_penalty=1.2
        )

        summaries = []
        
        # Prompt engineering for clinical notes
        system_prompt = """You are an expert Clinical Documentation Specialist and a Precise Data Extractor.

        GOAL: Extract and summarize only the key background data from the patient note chunk into a structured abstract.
        Do not repeat information.

        STRICT OUTPUT FORMAT:
        The output **MUST** contain **only** these five sections, populated with data from the note.

        Demographics: [Age, Sex]

        Medical History:
        - Past medical/surgical history
        - Medications
        - Allergies

        Family History:
        - Key hereditary conditions mentioned

        Social History:
        - Lifestyle factors, substance use, occupation

        Medication History:
        - Current and recent medications, including prescribed and non-prescribed use

        RULES AND CONSTRAINTS:
        1.  **SCOPE**: Focus exclusively on the five sections defined above. Do NOT include History of Present Illness, Physical Exam, or Diagnostics.
        2.  **FACTUALITY**: Do NOT infer, interpret, or create new information.
        3.  **EXTRACTION**: Extract only from what is explicitly stated in the patient note chunk.
        4.  **CONCISENESS**: Be concise and clinically precise.

                        """

        for i, chunk in enumerate(chunks):
            print(f"Processing chunk {i+1}/{len(chunks)}...")
            
            # Formatting strictly for chat/instruct models
            prompt = f"{system_prompt}\n\nPATIENT NOTE:\n{chunk}\n\nSUMMARY:"
            
            try:
                output = pipe(prompt)[0]['generated_text']
                # Basic parsing to remove input prompt from output
                clean_output = output.split("SUMMARY:")[-1].strip()
                summaries.append(clean_output)
            except Exception as e:
                print(f"Error processing chunk {i}: {e}")

        self._cleanup(model, tokenizer)
        return "\n".join(summaries)

    def meta_summarize(self, summary_a: str, summary_b: str, aggregator_model_id: str = "BioMistral/BioMistral-7B") -> str:
        """
        Takes the two summaries and fuses them.
        """
        print("--- Generating Meta-Summary ---")
        combined_input = f"Summary from Model A:\n{summary_a}\n\nSummary from Model B:\n{summary_b}"
        
        model, tokenizer = self._load_model(aggregator_model_id)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1000, return_full_text=False)
        
        prompt = f"""You are a Clinical Data API capable of merging medical summaries into structured JSON.

        INSTRUCTION:
        Synthesize the provided summaries into a single cohesive JSON object.
        
        STRICT OUTPUT SCHEMA:
        You must return a VALID JSON object with exactly these keys. Do not add keys. Do not add markdown formatting.

        {{
            "Demographics": "String containing Age and Sex",
            "History of Present Illness": ["List", "of", "key", "points"],
            "Medical History": ["List", "of", "history", "meds", "allergies"],
            "Family History": "String summary of hereditary conditions",
            "Social History": "String summary of lifestyle/substance use",
            "Physical Exam": "String summary of vitals and findings (or 'Not provided')",
            "Diagnostics": "String summary of labs/imaging (or 'Not provided')"
        }}

        RULES:
        1. Output must be parsable JSON.
        2. Escape all special characters inside strings.
        3. Do NOT include an Assessment or Plan.
        4. Do NOT include markdown ticks (```json). Just the raw JSON object.

        INPUT DATA:
        {combined_input}

        JSON OUTPUT:
        """
        
        output = pipe(prompt)[0]['generated_text']
        
        # --- FAIL-SAFE CLEANING LOGIC ---
        # If the prompt is still in the output, split by the last line of your prompt
        split_marker = 'JSON OUTPUT:'
        
        if split_marker in output:
            final_text = output.split(split_marker)[-1].strip()
        else:
            final_text = output.strip()
            
        self._cleanup(model, tokenizer)
        return final_text

In [ ]:
# 1. Data Prep
processor = ClinicalDataProcessor("dataset\\nbme-score-clinical-patient-notes\patient_notes.csv")

# Select a specific case number (e.g., 0)
CASE_ID = 0
raw_text = processor.get_case_notes(CASE_ID)

# Create chunks (Smart Chunking)
# If text is massive, chunk_size=2000 is safe for 7B models (usually 4096 ctx)
text_chunks = processor.smart_chunking(raw_text, chunk_size=2000, overlap_percentage=0.15)

In [ ]:
# 2. Summarization (The heavy lifting)
load_dotenv() 
MODEL_A = "BioMistral/BioMistral-7B"
# MODEL_A = 'google/medgemma-4b-it'
summarizerA = ClinicalSummarizer()

print(f"\n=== Summarizing with {MODEL_A} ===")
summary_a = summarizerA.summarize_chunks(text_chunks, MODEL_A)
print(f"Summary A generated (Length: {len(summary_a)})")

model_name_safe = MODEL_A.replace("/", "_")
output_path = f"{model_name_safe}_summary.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(summary_a, f, indent=4)

print(f"Summary JSON saved to: {output_path}")

In [ ]:
load_dotenv() 
MODEL_B = "m42-health/Llama3-Med42-8B"
summarizerB = ClinicalSummarizer()

print(f"\n=== Summarizing with {MODEL_B} ===")
summary_b = summarizerB.summarize_chunks(text_chunks, MODEL_B)
print(f"Summary A generated (Length: {len(summary_b)})")

model_name_safe = MODEL_B.replace("/", "_")
output_path = f"{model_name_safe}_summary.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(summary_b, f, indent=4)

print(f"Summary JSON saved to: {output_path}")

In [ ]:
file_path1 = "BioMistral_BioMistral-7B_summary.json"
file_path2 = "m42-health_Llama3-Med42-8B_summary.json"

load_dotenv() 
model = 'mistralai/Mistral-7B-Instruct-v0.3'
summary = ClinicalSummarizer()


with open(file_path1, "r", encoding="utf-8") as f:
    data1 = json.load(f)

with open(file_path2, "r", encoding="utf-8") as f:
    data2 = json.load(f)

final_summary = json.loads(summary.meta_summarize(data1, data2, model))

with open('final_summary_{CASE_ID}.json', "w", encoding="utf-8") as f:
    json.dump(final_summary, f, indent=4)


In [14]:
class MedicalSearchTool(BaseTool):
    name: str = "Medical Search Tool"
    description: str = "Useful for searching the web for clinical guidelines, medical definitions, and differential diagnoses."

    def _run(self, query: str) -> str:
        """
        CORRECT IMPLEMENTATION: Uses .text() instead of .invoke()
        """
        try:
            print(f"🔎 Searching for: {query}") # Debug print
            with DDGS() as ddgs:
                # CRITICAL FIX: Use .text(), not .invoke()
                results = list(ddgs.text(query, max_results=5))
                
                if not results:
                    return "No search results found for this query."
                
                # Format results nicely for the LLM
                formatted_results = []
                for r in results:
                    title = r.get('title', 'No Title')
                    link = r.get('href', 'No Link')
                    body = r.get('body', 'No Content')
                    formatted_results.append(f"Title: {title}\nLink: {link}\nSnippet: {body}")
                
                return "\n\n---\n\n".join(formatted_results)
                
        except Exception as e:
            return f"Error performing search: {str(e)}"


class DiagnosisCrew:
    def __init__(self):
        self.search_tool = MedicalSearchTool()
        self.llm = LLM(
            model="gemini/gemini-2.0-flash",
            api_key=os.getenv("GEMINI_API_KEY"),
            safety_settings=[
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"}
            ]
        )

    def run_diagnosis_search(self, clinical_summary: str):
        medical_researcher = Agent(
        role='Senior Medical Diagnostician',
        goal=(
            "You are a specialist medical researcher. "
            "Your task is NOT to provide definitive diagnoses, but to generate "
            "patient-specific differential diagnoses with risk stratification, "
            "clinical rationale, and recommended next steps based on trusted sources."
        ),
        backstory=(
            "You consult high-quality medical databases such as PubMed, "
            "NICE guidelines, Mayo Clinic, MedlinePlus. "
            "You integrate patient demographics, symptoms, lifestyle, medications, "
            "and family history into your reasoning."
        ),
            verbose=True,
            allow_delegation=False,
            tools=[self.search_tool],
            llm=self.llm,
        )

        research_task = Task(
            description=f"""
                Analyze the following Patient Case Summary:
                {clinical_summary}
                
                1. Extract key risk factors: age, medications, lifestyle, family history, comorbidities.
                2. Generate a **prioritized differential diagnosis**, explaining why each is likely.
                3. Identify **red flags** that need urgent attention.
                4. Suggest **next steps**, including: 
                - Recommended labs, imaging, or monitoring 
                - Lifestyle or medication modifications 
                - Specialist referrals if needed
                5. Include evidence from credible sources (PubMed, Mayo Clinic, MedlinePlus, NICE guidelines) 
                with citations or links.

                Return a structured report with the following sections:
                - Patient Summary
                - Risk Factors
                - Red Flags
                - Differential Diagnosis (prioritized)
                - Supporting Evidence
                - Recommended Next Steps

                Use the following websites
                https://www.mayoclinic.org/
                https://medlineplus.gov/
                https://pubmed.ncbi.nlm.nih.gov/
                https://familydoctor.org/
                https://www.nice.org.uk/
            """,
            expected_output=("A structured json formatted, patient-specific clinical report with evidence-based differential diagnoses, \
            risk assessment, and actionable next steps."),
            agent=medical_researcher,
            verbose=True
        )

        crew = Crew(
            agents=[medical_researcher],
            tasks=[research_task],
            process=Process.sequential,
            verbose=True
        )

        result = crew.kickoff()
        return result

In [15]:
print("\n=== Phase 3: Launching CrewAI Agent ===")
file_path3 = 'final_summary.json'
with open(file_path3, "r", encoding="utf-8") as f:
    summary_final = json.load(f)

crew_engine = DiagnosisCrew()
diagnosis_report = crew_engine.run_diagnosis_search(summary_final)


=== Phase 3: Launching CrewAI Agent ===


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 987de28d-dbdd-4762-afa1-172bf84aa14e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Medical Diagnostician                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│                  Analyze the following Patient Case Summary:                                                    │
│                  {'Demographics': '17-year-old male', 'History of Present Illness': ['Intermittent              │
│  palpitations over past 3-4 months without identifiable triggers; occurred most recently 2 days prior to        │
│  presentation, accompanied by transient shortness of breath which resolved upon cessation of activity.',        │
│  'Denies chest pain, diaphoresis, visual disturbances, syncope, or recent weight change.', 'Patient admits to   │
│  non-prescription use of Adderall multiple times weekly and regular consumption of caffeinated beverages        │
│  including coffee and energy drinks.'], 'Medical History': 'No significant past medical history reported.',     │
│  'Family History': 'Family history includes hyperthyroidism in both parents and paternal cardiac event within   │
│  recent memory.', 'Social History': 'Full-time college student majoring in Philosophy. Consumes alcohol         │
│  occasionally, specifically beer, typically on weekends. Never smoked cigarettes. Engages in recreational       │
│  marijuana use but infrequently. Actively participates in sports like basketball and running. Sexually active   │
│  with a long-term partner, utilizing condoms consistently for protection against sexually transmitted           │
│  diseases.', 'Physical Exam': 'Not provided as part of this summary.', 'Diagnostics': 'No specific lab results  │
│  or imaging studies were noted in the provided text.'}                                                          │
│                                                                                                                 │
│                  1. Extract key risk factors: age, medications, lifestyle, family history, comorbidities.       │
│                  2. Generate a **prioritized differential diagnosis**, explaining why each is likely.           │
│                  3. Identify **red flags** that need urgent attention.                                          │
│                  4. Suggest **next steps**, including:                                                          │
│                  - Recommended labs, imaging, or monitoring                                                     │
│                  - Lifestyle or medication modifications                                                        │
│                  - Specialist referrals if needed                                                               │
│                  5. Include evidence from credible sources (PubMed, Mayo Clinic, MedlinePlus, NICE guidelines)  │
│                  with citations or links.                                                                       │
│                                                                                                                 │
│                  Return a structured report with the following sections:                                        │
│                  - Patient Summary                                                                              │
│                  - Risk Factors                                                                                 │
│                  - Red Flags                                                                                    │
│                  - Differential Diagnosis (prioritized)

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Medical Diagnostician                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Okay, I will analyze the patient case summary, extract key information, generate a prioritized differential    │
│  diagnosis, identify red flags, suggest next steps, and provide supporting evidence from credible sources. I    │
│  will then return a structured JSON-formatted report.                                                           │
│                                                                                                                 │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "Patient Summary": {                                                                                         │
│      "Demographics": "17-year-old male",                                                                        │
│      "History of Present Illness": "Intermittent palpitations over the past 3-4 months without identifiable     │
│  triggers, most recently 2 days prior to presentation, accompanied by transient shortness of breath which       │
│  resolved upon cessation of activity. Denies chest pain, diaphoresis, visual disturbances, syncope, or recent   │
│  weight change. Reports non-prescription Adderall use multiple times weekly and regular consumption of          │
│  caffeinated beverages.",                                                                                       │
│      "Medical History": "No significant past medical history reported.",                                        │
│      "Family History": "Hyperthyroidism in both parents and paternal cardiac event within recent memory.",      │
│      "Social History": "Full-time college student. Occasional alcohol consumption (beer on weekends). Never     │
│  smoked. Infrequent recreational marijuana use. Actively participates in sports. Sexually active with a         │
│  long-term partner, using condoms consistently.",                                                               │
│      "Physical Exam": "Not provided.",                                                                          │
│      "Diagnostics": "No specific lab results or imaging studies were provided."                                 │
│    },                                                                                                           │
│    "Risk Factors": {                                                                                            │
│      "Age": "Adolescent age (17 years) increases the likelihood of certain conditions such as supraventricular  │
│  tachycardia (SVT).",                                                                                           │
│      "Medications/Substances": "Non-prescription Adderall use (stimulant) and regular caffeine consumption are  │
│  significant risk factors for palpitations and cardiac arrhythmias. Recreational marijuana use has potential    │
│  cardiovascular effects.",                                                                                      │
│      "Lifestyle": "Active participation in sports could be a trigger for palpitations if underlying cardiac     │
│  condition exists. Occasional alcohol consumption is a moderate risk factor.",                                  │
│      "Family History": "Family history of hyperthyroidi

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a94a5892-2537-4767-9276-630a3b390b61                                                                     │
│  Agent: Senior Medical Diagnostician                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 987de28d-dbdd-4762-afa1-172bf84aa14e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Okay, I will analyze the patient case summary, extract key information, generate a prioritized   │
│  differential diagnosis, identify red flags, suggest next steps, and provide supporting evidence from credible  │
│  sources. I will then return a structured JSON-formatted report.                                                │
│                                                                                                                 │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "Patient Summary": {                                                                                         │
│      "Demographics": "17-year-old male",                                                                        │
│      "History of Present Illness": "Intermittent palpitations over the past 3-4 months without identifiable     │
│  triggers, most recently 2 days prior to presentation, accompanied by transient shortness of breath which       │
│  resolved upon cessation of activity. Denies chest pain, diaphoresis, visual disturbances, syncope, or recent   │
│  weight change. Reports non-prescription Adderall use multiple times weekly and regular consumption of          │
│  caffeinated beverages.",                                                                                       │
│      "Medical History": "No significant past medical history reported.",                                        │
│      "Family History": "Hyperthyroidism in both parents and paternal cardiac event within recent memory.",      │
│      "Social History": "Full-time college student. Occasional alcohol consumption (beer on weekends). Never     │
│  smoked. Infrequent recreational marijuana use. Actively participates in sports. Sexually active with a         │
│  long-term partner, using condoms consistently.",                                                               │
│      "Physical Exam": "Not provided.",                                                                          │
│      "Diagnostics": "No specific lab results or imaging studies were provided."                                 │
│    },                                                                                                           │
│    "Risk Factors": {                                                                                            │
│      "Age": "Adolescent age (17 years) increases the likelihood of certain conditions such as supraventricular  │
│  tachycardia (SVT).",                                                                                           │
│      "Medications/Substances": "Non-prescription Adderall use (stimulant) and regular caffeine consumption are  │
│  significant risk factors for palpitations and cardiac arrhythmias. Recreational marijuana use has potential    │
│  cardiovascular effects.",                                                                                      │
│      "Lifestyle": "Active participation in sports could be a trigger for palpitations if underlying cardiac     │
│  condition exists. Occasional alcohol consumption is a

In [ ]:
with open("diagnosis_report.json", "w", encoding="utf-8") as f:
    f.write(str(diagnosis_report))
print("\n Diagnosis saved to 'diagnosis.md'")


 Diagnosis saved to 'diagnosis.md'


In [6]:
diagnosis_report

CrewOutput(raw=" It can sometimes happen after eating. This article will discuss some of the possible causes of shortness of breath after eating. It will also discuss when you should see a healthcare provider for your symptoms.\n\n---\n\nTitle: Tachycardia - Symptoms and causes - Mayo Clinic\nLink: https://www.mayoclinic.org/diseases-conditions/tachycardia/symptoms-causes/syc-20355127\nSnippet: Sep 13, 2023 ... Having certain medical conditions or living a certain lifestyle may put you at a higher risk of tachycardia. Risk factors include: · Anemia. · High blood pressure. · Mental health disorders, such as anxiety or stress. · Overuse of alcohol or caffeinated drinks. · Use of recreational drugs, such as cocaine. · Thyroid gland problems. · Smoking.\n\n---\n\nTitle: Palpitations | Michigan Medicine\nLink: https://www.uofmhealth.org/health-library/hw194830\nSnippet: Lifestyle Issues. Caffeine, alcohol, nicotine, or illegal drugs can cause palpitations . Medical Problems. Certain medical